## Daily Reddit API Export

### + pandas custom layer

In [ ]:
import json
import praw
import pandas as pd
import boto3
import time


s3 = boto3.client("s3")

def lambda_handler(event,context):
    
# specifying S3 bucket as import destination 

    bucket = "redditapibucket2"
    
# sending request to Reddit using acquired API keys

    reddit = praw.Reddit(
    client_id = "Kxke58vtbFOzeTO95IfkLQ",
    client_secret = "SCq6Gw9eoK5G8q4ylhqj46C3OT95sQ",
    user_agent = "mira")
    
# creating an empty list

    posts = []
    
# specifying subreddit information 

    uc_subreddit = reddit.subreddit("UkrainianConflict")
    
# specifying parameters for daily data export 

    for post in uc_subreddit.top(time_filter="day", limit = 150 ): 
        
# appending empty list with desired data

        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.created])
    
# transforming list to pandas DataFrame

    posts = pd.DataFrame(posts, columns=["title", "score", "id", "subreddit", "url", "num_comments", "timestamp"])

# saving data as JSON file 

    data = posts.to_json(orient = "index" , default_handler=str)

# specifyin gnaming convention to account for chaning naming convention each day

    timestr = time.strftime("%Y%m%d")
    
    filename= "redditapi" + timestr + ".json"
    
    uploadByteStream = bytes(json.dumps(data).encode("UTF-8"))

# saving JSON to S3 bucket

    s3.put_object(Bucket = bucket, Key = filename , Body = data)

# printing results of the operation 

    print("put complete")
